<a href="https://colab.research.google.com/github/PadariyaDebo/CGAN-DCGAN-WGAN/blob/main/PATEGAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/PadariyaDebo/mlforhealthlabpub.git

Cloning into 'mlforhealthlabpub'...
remote: Enumerating objects: 1192, done.
remote: Counting objects: 100% (128/128), done.
remote: Compressing objects: 100% (84/84), done.
remote: Total 1192 (delta 67), reused 48 (delta 44), pack-reused 1064
Receiving objects: 100% (1192/1192), 17.31 MiB | 8.64 MiB/s, done.
Resolving deltas: 100% (268/268), done.


In [2]:
!pip install numpy
!pip install argparse
!pip install pandas
!pip install tensorflow==1.15.0
!pip install scikit-learn
!pip install xgboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 412.3 MB 27 kB/s 
     |████████████████████████████████| 3.8 MB 56.5 MB/s 
     |████████████████████████████████| 50 kB 6.1 MB/s 
     |████████████████████████████████| 503 kB 65.6 MB/s 
  Created wheel for gast: filename=gast-0.2.2-py3-none-any.whl size=7554 sha256=78139d3353726abb418ccdf88d6eb934651193e8fb90608ea9551958d0f98f67
  Stored in directory: /root/.cache/pip/wheels/21/7f/02/420f32a803f7d0967b48dd823da3f558c5166991bfd204eef3
Successfully built gast
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.8.0
    Uninstalling tensorflow-estimator-2.8.0:
      Successfully uninstalled tensorflow-estimator-2.8.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.8.0
 

In [3]:
%cd /content/mlforhealthlabpub/alg/pategan
!pip install -v -e .

/content/mlforhealthlabpub/alg/pategan
Using pip 21.1.3 from /usr/local/lib/python3.7/dist-packages/pip (python 3.7)
Value for scheme.platlib does not match. Please report this to <https://github.com/pypa/pip/issues/9617>
distutils: /usr/local/lib/python3.7/dist-packages
sysconfig: /usr/lib/python3.7/site-packages
Value for scheme.purelib does not match. Please report this to <https://github.com/pypa/pip/issues/9617>
distutils: /usr/local/lib/python3.7/dist-packages
sysconfig: /usr/lib/python3.7/site-packages
Value for scheme.headers does not match. Please report this to <https://github.com/pypa/pip/issues/9617>
distutils: /usr/local/include/python3.7/UNKNOWN
sysconfig: /usr/include/python3.7m/UNKNOWN
Value for scheme.scripts does not match. Please report this to <https://github.com/pypa/pip/issues/9617>
distutils: /usr/local/bin
sysconfig: /usr/bin
Value for scheme.data does not match. Please report this to <https://github.com/pypa/pip/issues/9617>
distutils: /usr/local
sysconfig: /us

In [7]:
import numpy as np

def data_generator(no, dim, noise_rate):
  """Generate train/test dataset for PATE-GAN evaluation
  
  Args:
    no: The number of train/test samples
    dim: The number of dimensions in train/test features
    noise_rate: The amount of noise for generating labels
    
  Returns:
    train_data: Training data (feature + label)
    test_data: Testing data (feature + label)
  """
  
  # Define symmetric covariance matrix for generating features
  cov_matrix = np.random.uniform(0, 1, [dim, dim])
  cov_matrix = 0.5 * (cov_matrix + np.transpose(cov_matrix))
  
  # Generate train/test features
  x_train = np.random.multivariate_normal(np.zeros([dim,]), cov_matrix, [no,])
  x_test = np.random.multivariate_normal(np.zeros([dim,]), cov_matrix, [no,])
  
  # Define feature label relationship
  W = np.random.uniform(0, 1, [dim,])
  b = np.random.uniform(0, 1, 1)
  
  # Generate train/test labels
  y_train = np.matmul(x_train, W) + b + np.random.normal(0, noise_rate, no)
  y_train = np.reshape(1*(y_train), [-1, 1])
  
  y_test = np.matmul(x_test, W) + b + np.random.normal(0, noise_rate, no)
  y_test = np.reshape(1*(y_test), [-1, 1])
    
  train_data = np.concatenate((x_train, y_train), axis=1)
  test_data = np.concatenate((x_test, y_test), axis=1)
  
  # Normalization
  for i in range(dim+1):
    train_data[:, i] = train_data[:, i] - np.min(train_data[:, i])
    train_data[:, i] = train_data[:, i] / (np.max(train_data[:, i]) + 1e-8)
    
  # Normalization
  for i in range(dim+1):
    test_data[:, i] = test_data[:, i] - np.min(test_data[:, i])
    test_data[:, i] = test_data[:, i] / (np.max(test_data[:, i]) + 1e-8)
  
  return train_data, test_data

In [8]:
import numpy as np
from sklearn import metrics

# Predictive models
from sklearn.linear_model import LogisticRegression, PassiveAggressiveClassifier
from sklearn.naive_bayes import GaussianNB, BernoulliNB, MultinomialNB
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.ensemble import AdaBoostClassifier, BaggingClassifier, GradientBoostingClassifier
from xgboost import XGBRegressor


def supervised_model_training(x_train, y_train, x_test, 
                              y_test, model_name):
  """Train supervised learning models and report the results.
  
  Args:
    - x_train, y_train: training dataset
    - x_test, y_test: testing dataset
    - model_name: supervised model name such as logisticregression
    
  Returns:
    - auc: Area Under ROC Curve
    - apr: Average Precision and Recall
  """
     
  if model_name == 'logisticregression':
    model  = LogisticRegression()
  elif model_name == 'randomforest':      
    model = RandomForestClassifier()
  elif model_name == 'gaussiannb':  
    model = GaussianNB()
  elif model_name == 'bernoullinb':  
    model        = BernoulliNB()
  elif model_name == 'multinb':  
    model        = MultinomialNB()
  elif model_name == 'svmlin':         
    model        = svm.LinearSVC() 
  elif model_name == 'gbm':         
    model         = GradientBoostingClassifier()   
  elif model_name == 'Extra Trees':
    model =  ExtraTreesClassifier(n_estimators=20)
  elif model_name == 'LDA':
    model =  LinearDiscriminantAnalysis() 
  elif model_name == 'Passive Aggressive':
    model =   PassiveAggressiveClassifier()
  elif model_name == 'AdaBoost':
    model =   AdaBoostClassifier()
  elif model_name == 'Bagging':
    model =   BaggingClassifier()
  elif model_name == 'xgb':
    model =   XGBRegressor()                                
  
  if(model_name=='svmlin' or model_name=='Passive Aggressive'): 
    model.fit(x_train, y_train)
    predict = model.decision_function(x_test)
  elif (model_name =='xgb'):
    model.fit(np.asarray(x_train), y_train)
    predict = model.predict(np.asarray(x_test))
  else:
    model.fit(x_train, y_train)
    predict = model.predict_proba(x_test)[:,1]
        
  # AUC / AUPRC Computation
  auc = metrics.roc_auc_score(y_test, predict)
  apr = metrics.average_precision_score(y_test, predict)
  
  return auc, apr    

In [9]:
import tensorflow as tf
import numpy as np
import warnings
warnings.filterwarnings("ignore")

from sklearn.linear_model import LogisticRegression


def pate_lamda (x, teacher_models, lamda):
  '''Returns PATE_lambda(x).
  
  Args:
    - x: feature vector
    - teacher_models: a list of teacher models
    - lamda: parameter
    
  Returns:
    - n0, n1: the number of label 0 and 1, respectively
    - out: label after adding laplace noise.
  '''
      
  y_hat = list()
        
  for teacher in teacher_models:            
    temp_y = teacher.predict(np.reshape(x, [1,-1]))
    y_hat = y_hat + [temp_y]
  
  y_hat = np.asarray(y_hat)
  n0 = sum(y_hat == 0)
  n1 = sum(y_hat == 1)
  
  lap_noise = np.random.laplace(loc=0.0, scale=lamda)
  
  out = (n1+lap_noise) / float(n0+n1)
  out = int(out>0.5)
        
  return n0, n1, out 


def pategan(x_train, parameters):
  '''Basic PATE-GAN framework.
  
  Args:
    - x_train: training data
    - parameters: PATE-GAN parameters
      - n_s: the number of student training iterations
      - batch_size: the number of batch size for training student and generator
      - k: the number of teachers
      - epsilon, delta: Differential privacy parameters
      - lamda: noise size
      
  Returns:
    - x_train_hat: generated training data by differentially private generator
  '''
  
  # Reset the graph
  tf.reset_default_graph()
    
  # PATE-GAN parameters
  # number of student training iterations
  n_s = parameters['n_s']
  # number of batch size for student and generator training
  batch_size = parameters['batch_size']
  # number of teachers
  k = parameters['k']
  # epsilon
  epsilon = parameters['epsilon']
  # delta
  delta = parameters['delta']
  # lamda
  lamda = parameters['lamda']
  
  # Other parameters
  # alpha initialize
  L = 20
  alpha = np.zeros([L])
  # initialize epsilon_hat
  epsilon_hat = 0
    
  # Network parameters
  no, dim = x_train.shape
  # Random sample dimensions
  z_dim = int(dim)
  # Student hidden dimension
  student_h_dim = int(dim)
  # Generator hidden dimension
  generator_h_dim = int(4*dim)  
  
  ## Partitioning the data into k subsets
  x_partition = list()
  partition_data_no = int(no/k)
    
  idx = np.random.permutation(no)
    
  for i in range(k):
    temp_idx = idx[int(i*partition_data_no):int((i+1)*partition_data_no)]
    temp_x = x_train[temp_idx, :]      
    x_partition = x_partition + [temp_x]    
  
  ## Necessary Functions for buidling NN models
  # Xavier Initialization Definition
  def xavier_init(size):
    in_dim = size[0]
    xavier_stddev = 1. / tf.sqrt(in_dim / 2.)
    return tf.random_normal(shape = size, stddev = xavier_stddev)    
        
  # Sample from uniform distribution
  def sample_Z(m, n):
    return np.random.uniform(-1., 1., size = [m, n])
     
  ## Placeholder
  # PATE labels
  Y = tf.placeholder(tf.float32, shape = [None, 1])  
  # Random Variable    
  Z = tf.placeholder(tf.float32, shape = [None, z_dim])
   
  ## NN variables   
  # Student
  S_W1 = tf.Variable(xavier_init([dim, student_h_dim]))
  S_b1 = tf.Variable(tf.zeros(shape=[student_h_dim]))
    
  S_W2 = tf.Variable(xavier_init([student_h_dim,1]))
  S_b2 = tf.Variable(tf.zeros(shape=[1]))

  theta_S = [S_W1, S_W2, S_b1, S_b2]
    
  # Generator

  G_W1 = tf.Variable(xavier_init([z_dim, generator_h_dim]))
  G_b1 = tf.Variable(tf.zeros(shape=[generator_h_dim]))

  G_W2 = tf.Variable(xavier_init([generator_h_dim,generator_h_dim]))
  G_b2 = tf.Variable(tf.zeros(shape=[generator_h_dim]))

  G_W3 = tf.Variable(xavier_init([generator_h_dim,dim]))
  G_b3 = tf.Variable(tf.zeros(shape=[dim]))
    
  theta_G = [G_W1, G_W2, G_W3, G_b1, G_b2, G_b3]

  ## Models
  def generator(z):
    G_h1 = tf.nn.tanh(tf.matmul(z, G_W1) + G_b1)
    G_h2 = tf.nn.tanh(tf.matmul(G_h1, G_W2) + G_b2)
    G_out = tf.nn.sigmoid(tf.matmul(G_h2, G_W3) + G_b3)
        
    return G_out
    
  def student(x):
    S_h1 = tf.nn.relu(tf.matmul(x, S_W1) + S_b1)
    S_out = tf.matmul(S_h1, S_W2) + S_b2
        
    return S_out
      
  ## Loss  
  G_sample = generator(Z)
  S_fake = student(G_sample)
  
  S_loss = tf.reduce_mean(Y * S_fake) - tf.reduce_mean((1-Y) * S_fake)
  G_loss = -tf.reduce_mean(S_fake)
  
  # Optimizer
  S_solver = (tf.train.RMSPropOptimizer(learning_rate=1e-4)
              .minimize(-S_loss, var_list=theta_S))
  G_solver = (tf.train.RMSPropOptimizer(learning_rate=1e-4)
              .minimize(G_loss, var_list=theta_G))
  
  clip_S = [p.assign(tf.clip_by_value(p, -0.01, 0.01)) for p in theta_S]
  
  ## Sessions
  sess = tf.Session()
  sess.run(tf.global_variables_initializer())
        
  ## Iterations
  while epsilon_hat < epsilon:      
          
    # 1. Train teacher models
    teacher_models = list()
    
    for _ in range(k):
                
      Z_mb = sample_Z(partition_data_no, z_dim)
      G_mb = sess.run(G_sample, feed_dict = {Z: Z_mb})
                
      temp_x = x_partition[i]
      idx = np.random.permutation(len(temp_x[:, 0]))
      X_mb = temp_x[idx[:partition_data_no], :]
                
      X_comb = np.concatenate((X_mb, G_mb), axis = 0)
      Y_comb = np.concatenate((np.ones([partition_data_no,]), 
                               np.zeros([partition_data_no,])), axis = 0)
                
      model = LogisticRegression()
      model.fit(X_comb, Y_comb)
      teacher_models = teacher_models + [model]
            
    # 2. Student training
    for _ in range(n_s):
          
      Z_mb = sample_Z(batch_size, z_dim)
      G_mb = sess.run(G_sample, feed_dict = {Z: Z_mb})
      Y_mb = list()
            
      for j in range(batch_size):                
        n0, n1, r_j = pate_lamda(G_mb[j, :], teacher_models, lamda)
        Y_mb = Y_mb + [r_j]
       
        # Update moments accountant
        q = np.log(2 + lamda * abs(n0 - n1)) - np.log(4.0) - \
            (lamda * abs(n0 - n1))
        q = np.exp(q)
                
        # Compute alpha
        for l in range(L):
          temp1 = 2 * (lamda**2) * (l+1) * (l+2)
          temp2 = (1-q) * ( ((1-q)/(1-q*np.exp(2*lamda)))**(l+1) ) + \
                  q * np.exp(2*lamda * (l+1))
          alpha[l] = alpha[l] + np.min([temp1, np.log(temp2)])
        
      # PATE labels for G_mb  
      Y_mb = np.reshape(np.asarray(Y_mb), [-1,1])
                
      # Update student
      _, D_loss_curr, _ = sess.run([S_solver, S_loss, clip_S], 
                                   feed_dict = {Z: Z_mb, Y: Y_mb})
    
    # Generator Update        
    Z_mb = sample_Z(batch_size, z_dim)
    _, G_loss_curr = sess.run([G_solver, G_loss], feed_dict = {Z: Z_mb})
        
    # epsilon_hat computation
    curr_list = list()        
    for l in range(L):
      temp_alpha = (alpha[l] + np.log(1/delta)) / float(l+1)
      curr_list = curr_list + [temp_alpha]
        
    epsilon_hat = np.min(curr_list)    

  ## Outputs
  x_train_hat = sess.run([G_sample], feed_dict = {Z: sample_Z(no, z_dim)})[0]
    
  return x_train_hat

In [17]:
import tensorflow as tf
import tensorflow.compat.v1 as tf
tf.disable_eager_execution()
tf.disable_v2_behavior()


from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import argparse
import numpy as np
import pandas as pd
import sys
sys.argv=['']
del sys
from data_generator import data_generator
from utils import supervised_model_training
from pate_gan import pategan
from sklearn.preprocessing import MinMaxScaler

#%% 
def pategan_main (args):
  """PATEGAN Main function.
  
  Args:
    data_no: number of generated data
    data_dim: number of data dimensions
    noise_rate: noise ratio on data
    iterations: number of iterations for handling initialization randomness
    n_s: the number of student training iterations
    batch_size: the number of batch size for training student and generator
    k: the number of teachers
    epsilon, delta: Differential privacy parameters
    lamda: noise size
    
  Returns:
    - results: performances of Original and Synthetic performances
    - train_data: original data
    - synth_train_data: synthetically generated data
  """
  
  # Supervised model types
  models = ['logisticregression','randomforest', 'gaussiannb','bernoullinb',
            'svmlin', 'Extra Trees','LDA', 'AdaBoost','Bagging','gbm', 'xgb']
  
  # Data generation
  if args.dataset == 'random':
      train_data, test_data = data_generator(args.data_no, args.data_dim, 
                                         args.noise_rate)
      data_dim = args.data_dim
  elif args.dataset == 'credit':
      # Insert relevant dataset here, and scale between 0 and 1.
      data = pd.read_csv('creditcard.csv').to_numpy()
      data = MinMaxScaler().fit_transform(data)
      train_ratio = 0.5
      train = np.random.rand(data.shape[0])<train_ratio 
      train_data, test_data = data[train], data[~train]
      data_dim = data.shape[1]
  
  # Define outputs
  results = np.zeros([len(models), 4])
  
  # Define PATEGAN parameters
  parameters = {'n_s': args.n_s, 'batch_size': args.batch_size, 'k': args.k, 
                'epsilon': args.epsilon, 'delta': args.delta, 
                'lamda': args.lamda}
  
  # Generate synthetic training data
  best_perf = 0.0
  
  for it in range(args.iterations):
    print('Iteration',it)
    synth_train_data_temp = pategan(train_data, parameters)
    temp_perf, _ = supervised_model_training(
        synth_train_data_temp[:, :(data_dim-1)], 
        np.round(synth_train_data_temp[:, (data_dim-1)]),
        train_data[:, :(data_dim-1)], 
        np.round(train_data[:, (data_dim-1)]),
        'logisticregression')
    
    # Select best synthetic data
    if temp_perf > best_perf:
      best_perf = temp_perf.copy()
      synth_train_data = synth_train_data_temp.copy()
      
    print('Iteration: ' + str(it+1))
    print('Best-Perf:' + str(best_perf))
  
  # Train supervised models
  for model_index in range(len(models)):
    model_name = models[model_index]
    
    # Using original data
    results[model_index, 0], results[model_index, 2] = (
        supervised_model_training(train_data[:, :(data_dim-1)], 
                                  np.round(train_data[:, (data_dim-1)]),
                                  test_data[:, :(data_dim-1)], 
                                  np.round(test_data[:, (data_dim-1)]),
                                  model_name))
        
    # Using synthetic data
    results[model_index, 1], results[model_index, 3] = (
        supervised_model_training(synth_train_data[:, :(data_dim-1)], 
                                  np.round(synth_train_data[:, (data_dim-1)]),
                                  test_data[:, :(data_dim-1)], 
                                  np.round(test_data[:, (data_dim-1)]),
                                  model_name))

    
    
  # Print the results for each iteration
  results = pd.DataFrame(np.round(results, 4), 
                         columns=['AUC-Original', 'AUC-Synthetic', 
                                  'APR-Original', 'APR-Synthetic'])
  print(results)
  print('Averages:')
  print(results.mean(axis=0))
  
  return results, train_data, synth_train_data

  
#%%  
if __name__ == '__main__':
  
  # Inputs for the main function
  parser = argparse.ArgumentParser()
  parser.add_argument(
      '--data_no',
      help='number of generated data',
      default=10000,
      type=int)
  parser.add_argument(
      '--data_dim',
      help='number of dimensions of generated dimension (if random)',
      default=10,
      type=int)
  parser.add_argument(
      '--dataset',
      help='dataset to use',
      default='random',
      type=str)
  parser.add_argument(
      '--noise_rate',
      help='noise ratio on data',
      default=1.0,
      type=float)
  parser.add_argument(
      '--iterations',
      help='number of iterations for handling initialization randomness',
      default=50,
      type=int)
  parser.add_argument(
      '--n_s',
      help='the number of student training iterations',
      default=1,
      type=int)
  parser.add_argument(
      '--batch_size',
      help='the number of batch size for training student and generator',
      default=64,
      type=int)
  parser.add_argument(
      '--k',
      help='the number of teachers',
      default=10,
      type=float)
  parser.add_argument(
      '--epsilon',
      help='Differential privacy parameters (epsilon)',
      default=1.0,
      type=float)
  parser.add_argument(
      '--delta',
      help='Differential privacy parameters (delta)',
      default=0.00001,
      type=float)
  parser.add_argument(
      '--lamda',
      help='PATE noise size',
      default=1.0,
      type=float)
  
  
  args = parser.parse_args() 
  parser.add_argument('-f')

  
  # Calls main function  
  results, ori_data, synth_data = pategan_main(args)

Iteration 0



Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Iteration: 1
Best-Perf:0.5598131449177359
Iteration 1
Iteration: 2
Best-Perf:0.7436221842059335
Iteration 2
Iteration: 3
Best-Perf:0.7436221842059335
Iteration 3
Iteration: 4
Best-Perf:0.7436221842059335
Iteration 4
Iteration: 5
Best-Perf:0.7995233055568115
Iteration 5
Iteration: 6
Best-Perf:0.7995233055568115
Iteration 6
Iteration: 7
Best-Perf:0.7995233055568115
Iteration 7
Iteration: 8
Best-Perf:0.7995233055568115
Iteration 8
Iteration: 9
Best-Perf:0.7995233055568115
Iteration 9
Iteration: 10
Best-Perf:0.7995233055568115
Iteration 10
Iteration: 11
Best-Perf:0.7995233055568115
Iteration 11
Iteration: 12
Best-Perf:0.7995233055568115
Iteration 12
Iteration: 13
Best-Perf:0.7995233055568115
Iteration 13
Iteration: 14
Best-Perf:0.7995233055568115
Iteration 14
Iteration: 15
Best-Perf:0.7995233055568115
Iteration 15
Iteration: 16
Best-Perf:0.7995233055568115
Iteration 16
Iteration: 17
Best-Perf:0.7995233055568115
Iteration 17
Iteration: 18
Best-Perf:0.7995233055568115
Iteration 18
Iteration:

In [16]:
%tb

SystemExit: ignored

In [11]:
%tb

SystemExit: ignored